In [1]:
# Import required libraries  
import os 
import openai
from dotenv import dotenv_values
from azure.core.credentials import AzureKeyCredential
from langchain.chat_models import AzureChatOpenAI
from azure.search.documents import SearchClient

from chatFunctions import chatBot

In [2]:
env_name = "../../llm.env" # change to use your own .env file
config = dotenv_values(env_name)

#Azure OpenAI
openai.api_type = config["OPENAI_API_TYPE"] #"azure"
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE']
openai.api_version = config['OPENAI_API_VERSION']

## Cog Search
cogsearch_name = config["COGSEARCH_NAME"] #TODO: fill in your cognitive search name
index_name = config["COGSEARCH_INDEX_NAME"] #TODO: fill in your index name: must only contain lowercase, numbers, and dashes
key = config["COGSEARCH_API_KEY"] #TODO: fill in your api key with admin key
service_endpoint = "https://"+config["COGSEARCH_NAME"] + ".search.windows.net"

credential = AzureKeyCredential(key)

## Chatbot

In [3]:
engine = "gpt-4-32k"
#engine = "gpt-35-turbo"
llm = AzureChatOpenAI(
    deployment_name=engine,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
    temperature=0.0, verbose = True
)

## ACS 
numChunks = 10
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

## llm
human_query = " earning calls for Target"

template_qa_chain = """You are a chatbot having a conversation with a human. 
        Given the Context, Chat History, and Human Query, 
        answer without hallucinating. If you don't have the answer say "I don't have the answer" """

template_context_summarization =  """
        Summarize the context so it includes the details related to the human query. """

max_token_for_context = 16000


cb = chatBot(llm, search_client, max_token_for_context = max_token_for_context, 
                 template_qa_chain = template_qa_chain, template_context_summarization = template_context_summarization, 
                 numChunks= numChunks, vectorColName = "contentVector",
                 to_debug = False)

## Query 1

In [4]:
query = "How many small stores was there in transcript for Ticker TGT for year 2021 and quarter Q4?"
cb.run(query)



'In the transcript for Ticker TGT for the year 2021 and quarter Q4, it is mentioned that Target Corporation had 140 small format stores. They plan to add another 30 to 40 per year over the next several years.'

## Query 2

In [5]:
query = "Is there any details about expense rate for Ticker TGT for year 2021 and quarter Q4 "
cb.run(query)

'In the provided context, there are no specific details about the expense rate for Target Corporation (Ticker: TGT) for the year 2021 and quarter Q4. However, it is mentioned that they expect their 2021 SG&A expense rate to be lower than their 2019 rate of 20.8%.'

## Check memory

In [6]:
cb.qa_chain.memory.chat_memory

ChatMessageHistory(messages=[HumanMessage(content='How many small stores was there in transcript for Ticker TGT for year 2021 and quarter Q4?', additional_kwargs={}, example=False), AIMessage(content='In the transcript for Ticker TGT for the year 2021 and quarter Q4, it is mentioned that Target Corporation had 140 small format stores. They plan to add another 30 to 40 per year over the next several years.', additional_kwargs={}, example=False), HumanMessage(content='Is there any details about expense rate for Ticker TGT for year 2021 and quarter Q4 ', additional_kwargs={}, example=False), AIMessage(content='In the provided context, there are no specific details about the expense rate for Target Corporation (Ticker: TGT) for the year 2021 and quarter Q4. However, it is mentioned that they expect their 2021 SG&A expense rate to be lower than their 2019 rate of 20.8%.', additional_kwargs={}, example=False)])